In [3]:
from multiprocessing import Pool
import itertools  
from sys import platform as sys_pf
import matplotlib
matplotlib.use("TkAgg")
import os, sys, time, pickle
#if sys_pf == 'darwin':
#    import matplotlib
#    matplotlib.use("TkAgg")
#    import os, sys, time, pickle
cwd = os.getcwd()
#import os, sys, time, pickle
cwd = os.getcwd()
sys.path.insert(0,cwd+'/F3')
from F3 import F2_mov as F2
from F3 import F3_mat as F3mat
from F3 import K2i_mat as K2
from F3 import G_mov as Gm
import defns
import math
import scipy
import numpy as np
import time
from numba import jit,njit,prange
from F3 import sums_mov as sums
from scipy.optimize import fsolve
from defns import chop
from scipy import interpolate


#@njit(fastmath=True)
def kcot_2par(qk2,params):
    a0 = params[0]
    r0 = params[1]
    return 1/a0 + r0*qk2/2.0

#@njit(fastmath=True)
def kcot_const(qk2,params):
    a0 = params[0]
    return 1/a0 
    
    
#@njit(fastmath=True)
def kiso_pole(Ecm_m,params):
    c = params[0]
    M_m = params[1]
    return c/(Ecm_m**2-M_m**2);

  
#@njit(fastmath=True)
def kiso_const(Ecm_m,params):
    c = params[0]
    return c;

def write_db_Fmat00():
    F3mat.write_db_Fmat00()


def QC3(e, L, nnP, kcot_in, params_kcot, kiso, params_kiso ):

    kcot=lambda qk2 : kcot_in(qk2,params_kcot)

    F00 = F3mat.F3mat00(e,L,0.5,nnP,kcot)
    
    ecm = np.sqrt(e**2 - (sums.norm(nnP)*2*math.pi/L)**2 )
    Delta = (ecm**2 -9.0)/9.0
    Kiso = kiso(ecm, params_kiso)
    
    ones = np.ones(len(F00))
    Fiso = 1/(ones@F00@ones)
    return Fiso  + Kiso
#     return Fiso
    
    
def find_sol(Estart, Eend, steps,L,nnP,kcot,params_kcot,kiso,params_kiso):
    #start = time.time()
    energies = np.linspace(Estart, Eend,steps)
    param = []
    for i in range(steps):
        param.append((energies[i],L,nnP,kcot,params_kcot ,kiso,params_kiso))
    #print('param', param[0])
    #print('E=',Estart,Eend)
    res = list(itertools.starmap(QC3, param))
    #print('Energies', energies)
    #print('QC3iso', res)
    func = interpolate.InterpolatedUnivariateSpline(energies, res)

    
    f1=func(Estart)
    f2=f1
    Eend=Estart
    iter=0
    while (f1*f2>=0 and iter<steps*2 ):
        Eend+= abs(Eend-Estart)/(steps*2) 
        f2=func(Eend  )
        iter+=1
    
    E3=scipy.optimize.fsolve(func, 0.5*(Estart+Eend) )
    #print('Estart=',Estart,'Eend=', Eend, 'Esol=',E3)
    #end = time.time()
    #print('python time:', end - start, ' s')
    write_db_Fmat00()    
    return E3

def find_sol_check(Estart, Eend, steps,L,nnP,kcot,params_kcot,kiso,params_kiso):
    energies = np.linspace(Estart, Eend,steps)
    param = []
    for i in range(steps):
        param.append((energies[i],L,nnP,kcot,params_kcot ,kiso,params_kiso))
    res = list(itertools.starmap(QC3, param))
    func = interpolate.InterpolatedUnivariateSpline(energies, res)
    E3=scipy.optimize.fsolve(func, 0.5*(Estart+Eend) )[0]

    #refining
    eps=1
    E3_to_check=E3
    delta= abs(Eend-Estart)/2.0
    #print("find_sol_check:",energies, E3_to_check )
    iter=0
    while (eps>1e-1 and iter<6):
        if (E3_to_check-delta<Estart):
            Es=Estart
        else:
            Es=E3_to_check-delta
        energies = np.linspace(Es, (E3_to_check+delta),steps)
        param = []
        #print("find_sol_check:",energies)

        for i in range(steps):
            param.append((energies[i],L,nnP,kcot,params_kcot ,kiso,params_kiso))
        res = list(itertools.starmap(QC3, param))
        func = interpolate.InterpolatedUnivariateSpline(energies, res)
        E3_check=scipy.optimize.fsolve(func, 0.5*(Estart+Eend) )[0]
       # print("find_sol_check:",Es, E3_to_check+delta, E3_check )

        eps=abs(E3_check-E3_to_check)
        E3_to_check=E3_check
        iter+=1

    E3=E3_to_check
    
    write_db_Fmat00()    
    return E3
    

def find_sol_bisect(Estart, Eend,  steps,L,nnP,kcot,params_kcot,kiso,params_kiso):
    
    func=lambda e : QC3(e ,L,nnP,kcot,params_kcot ,kiso,params_kiso)
    f1=func(Estart)
    f2=func(Eend)
    if (f1*f2>=0):
        return -10


    ##print("   bracket=[ ",Estart," ,",Eend," ] -> [",func(Estart)," , ",func(Eend),"]" )
    E3=scipy.optimize.root_scalar(func,method='brentq',bracket=[Estart,Eend], x0=0.5*(Estart+Eend), xtol=1e-3 )  #  bisect brentq
    ##print(" QC3 converged= ",E3.converged , "in ",E3.iterations, " iteration" ,  "result =", E3.root )
    if (not E3.converged):
        print("error not coverged")
        return -10
    if (E3.root<Estart or E3.root>Eend):
        print("E3.root=",E3.root ,"out of bound [",Estart,",",Eend,"]" )
        return -10
    write_db_Fmat00()    
    return E3.root


found database file
reading a database of dimension= 7211
found database files
reading a database of dimension= 158930


In [40]:
import plotly.express as px
fig = px.line(  title='QC3-pole')
params_kcot=[10, 0]

cs=[-100,-10,10,100]
NL=30

Li=np.zeros((len(cs),NL))
r_pole=np.zeros((len(cs),2,NL))
for ic in range(len(cs)):


    params_kiso=[cs[ic], 3.5]
    
    L=4.5
    m= 1
    #E1=0.35
    #E3=0.38
    delta=0.02
    delta=0.04
    steps=4
    Lm=L*m


    em=[]
    nnP=np.array([0.0,0.0,0.0])


    for i in range(0,NL):
        
        Ei=3.451-i*delta*0.5
        Ef=3.461-i*delta*0.5
        found=0
        #out=find_sol_check(Ei, Ef, steps,Lm,nnP,kcot_2par,params_kcot, kiso_const,params_kiso)
        #out=find_sol(Ei, Ef, steps,Lm,nnP,kcot_2par,params_kcot, kiso_pole,params_kiso)[0]
        out=find_sol_bisect(Ei, Ef, steps,Lm,nnP,kcot_2par,params_kcot, kiso_pole,params_kiso)
        #print("i=",i," sol=",out)
        if (out>0 and out < 5 ):
            r_pole[ic][found][i]=(out)
            found+=1
        
        
        while found<2:
            Ei+=delta
            Ef+=delta
            #out_tmp=find_sol_check(Ei, Ef, steps,Lm,nnP,kcot_2par,params_kcot, kiso_const,params_kiso)
            #out_tmp=find_sol(Ei, Ef, steps,Lm,nnP,kcot_2par,params_kcot, kiso_pole,params_kiso)[0]
            out_tmp=find_sol_bisect(Ei, Ef, steps,Lm,nnP,kcot_2par,params_kcot, kiso_pole,params_kiso)
            #print("i=",i, "Erange=[",Ei,",",Ef,"]  " ," sol=",out_tmp)
            if (abs(out-out_tmp)>1e-1 and out_tmp>0):
                out=out_tmp
                r_pole[ic][found][i]=(out)
                found+=1
        #print(Lm, r_pole[0][len(r_pole[0])-1], r_pole[1][len(r_pole[1])-1]  )       
        Li[ic][i]=(Lm)
        Lm=Lm+0.04   
        

    
    for r1 in r_pole[ic]:
        fig.add_scatter(x=Li[0], y=r1, mode = 'lines+markers',name='c={0}'.format(cs[ic])  )

fig.show()   

computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
writing database
computing new value: 
computing new value: 
computing new value: 
writing database
computing new value: 
computing new value: 
writing database
computing new value: 
computing new value: 
writing database
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
computing new value: 
writing database
computing new value: 
computing new value: 
writing datab

In [38]:
c='c={0}'.format(2)
c

'c=2'